In [2]:
import pyspark.sql.functions as F
from credentials import MY_CREDENTIALS
from data_location import DELTA_LOCATION

from spark_bi.spark import FutPathlingContext

pc = FutPathlingContext.create(
    app_name="example-spark-app", hadoop_config=MY_CREDENTIALS.to_hadoop_config()
)
delta_lake = pc.read.delta(DELTA_LOCATION)

:: loading settings :: url = jar:file:/Users/mabe/Git/spark-bi/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/mabe/.ivy2.5.2/cache
The jars for the packages stored in: /Users/mabe/.ivy2.5.2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-012f4abf-f4d7-4917-9d82-0cff792215c9;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;9.1.0 in local-m2-cache
	found io.delta#delta-spark_2.13;4.0.0 in local-m2-cache
	found io.delta#delta-storage;4.0.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.13.1 in local-m2-cache
:: resolution report :: resolve 87ms :: artifacts dl 5ms
	:: modules in use:
	au.csiro.pathling#library-runtime;9.1.0 from local-m2-cache in [default]
	io.delta#delta-spark_2.13;4.0.0 from local-m2-cache in [default]
	io.delta#delta-storage;4.0.0 fr

For at finde anvenderløsning bruger vi coexistence-tags der er beskrevet i: https://ehealth-dk.atlassian.net/wiki/spaces/EDTW/pages/2355986433/Multitenancy



In [10]:
from pyspark.sql.functions import col

video_appointments = (
    delta_lake.view(
        resource="Appointment",
        select=[
            {
                "column": [
                    {"name": "app_id", "path": "getResourceKey()"},
                    {"name": "profile", "path": "meta.profile.first()"},
                    {"name": "scheduled_start", "path": "start"},
                    {"name": "scheduled_end", "path": "end"},
                ]
            }
        ],
    )
    .filter(
        F.col("profile")
        == "http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-group-videoappointment"
    )
    .withColumn(
        "duration_minutes",
        (F.unix_timestamp("scheduled_end") - F.unix_timestamp("scheduled_start")) / 60,
    )
)
video_appointments.head(5)

[Row(app_id='Appointment/1000341100', profile='http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-group-videoappointment', scheduled_start=datetime.datetime(2022, 9, 14, 9, 8, 35, 99000), scheduled_end=datetime.datetime(2022, 9, 14, 9, 8, 35, 99000), duration_minutes=0.0),
 Row(app_id='Appointment/1000342024', profile='http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-group-videoappointment', scheduled_start=datetime.datetime(2022, 9, 17, 9, 7, 53, 855000), scheduled_end=datetime.datetime(2022, 9, 17, 9, 7, 53, 855000), duration_minutes=0.0),
 Row(app_id='Appointment/1000342640', profile='http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-group-videoappointment', scheduled_start=datetime.datetime(2022, 9, 19, 9, 8, 21, 70000), scheduled_end=datetime.datetime(2022, 9, 19, 9, 8, 21, 70000), duration_minutes=0.0),
 Row(app_id='Appointment/1000343880', profile='http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-group-videoappointment', scheduled_start=datet

In [13]:
(
    video_appointments.groupby("duration_minutes")
    .agg(F.count("app_id").alias("n_appointments"))
    .head(5)
)

[Row(duration_minutes=0.0, n_appointments=2822)]

Desværre er der ikke nogle videomøder på TRIFORK-miljøet med en planlagt varighed over 0 minutter.